In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.typing as npt
from astroML.correlation import two_point

from stellar_stream import StellarStream


In [ ]:
size = 50000
mu, phi2_sigma = 0, 1 # mean and standard deviation
vlos_sigma = 5
rng = np.random.default_rng(seed=42)
phi1 = rng.uniform(-180, 180, size)
phi2 = rng.normal(mu, phi2_sigma, size)
vlos = rng.normal(mu, vlos_sigma, size)


def wiggle(location, amplitude, width):
    return amplitude * np.exp(-0.5*((phi1-location)/width)**2)



def gap(phi1, phi2, vlos, gap_center, gap_depth, gap_width):
    prob_keep = 1.0 - gap_depth * np.exp(-0.5 * ((phi1 - gap_center)/gap_width)**2)
    mask = np.random.rand(len(phi1)) < prob_keep
    return phi1[mask], phi2[mask], vlos[mask]



def add_substructure(phi1, phi2, vlos, scale, amount):
    phi1_copy, phi2_copy, vlos_copy = phi1.copy(), phi2.copy(), vlos.copy()
    for i in range(amount):
        location = rng.uniform(min(phi1), max(phi1), 1)
        depth = rng.uniform(0.1, 1, 1)
        width = rng.lognormal(mean=np.log(scale), sigma=0.3)
        phi1_copy, phi2_copy, vlos_copy = gap(phi1_copy, phi2_copy, vlos_copy, location, depth, width)
    return phi1_copy, phi2_copy, vlos_copy



In [ ]:
def inject_substructure(phi1, phi2=None, vlos=None,
                        amount=1, scale=0.5,  # typical width scale (same units as phi1)
                        mode="point",         # "point" or "binned"
                        kind="gaussian",      # currently only gaussian supported
                        depths=None,          # array-like or scalar: depletion fraction (0..1) for gaps OR amplitude for overdensity
                        amps=None,            # for overdensity amplitude (multiplicative)
                        rng=None,
                        binned_kwargs=None):
    """
    Inject `amount` substructures into a stream along phi1.

    Parameters
    ----------
    phi1 : 1D array
        along-stream coordinate of points (assumed in same units as `scale`).
    phi2 : 1D array or None
        across-stream coordinate (will be carried through unchanged except for duplication/removal).
    vlos : 1D array or None
        velocities (carried through).
    amount : int
        number of features to inject.
    scale : float
        nominal width (sigma) for gaussian perturbations (same units as phi1).
    mode : {"point","binned"}
        - "point": modify the point catalog by probabilistic removal (gaps) or Poisson duplications (overdensities).
        - "binned": expects binned_kwargs dict with keys (bins, density) and returns modified density.
    depths : scalar or array-like
        For gaps: fraction removed at center (0..1). If None, uniform random in [0.1,1.0].
    amps : scalar or array-like
        For overdensities: fractional amplitude (e.g. 0.5 means +50% at center). If None, drawn from Uniform(0.1,1.0).
    rng : np.random.Generator or None
        RNG for reproducibility. If None, uses np.random.default_rng().
    binned_kwargs : dict
        Required if mode == "binned". Should contain 'bins' (edges) and 'density' (array).
    
    Returns
    -------
    If mode == "point":
      phi1_new, phi2_new, vlos_new, injected_params
    If mode == "binned":
      density_new, injected_params

    injected_params: list of dicts {kind, center, sigma, depth/amp}
    """
    if rng is None:
        rng = np.random.default_rng()

    phi1 = np.asarray(phi1)
    N = len(phi1)
    has_phi2 = phi2 is not None
    has_vlos = vlos is not None
    if has_phi2:
        phi2 = np.asarray(phi2)
    if has_vlos:
        vlos = np.asarray(vlos)

    # Helpers to draw parameters
    def draw_centers(n):
        return rng.uniform(phi1.min(), phi1.max(), size=n)

    def draw_depths(n):
        if depths is None:
            return rng.uniform(0.1, 1.0, size=n)
        d = np.asarray(depths)
        return np.broadcast_to(d, (n,)) if d.size == 1 else d

    def draw_amps(n):
        if amps is None:
            return rng.uniform(0.1, 1.0, size=n)
        a = np.asarray(amps)
        return np.broadcast_to(a, (n,)) if a.size == 1 else a

    centers = draw_centers(amount)
    sigmas = rng.lognormal(mean=np.log(scale), sigma=0.3, size=amount)  # lognormal scatter
    depths_arr = draw_depths(amount)
    amps_arr = draw_amps(amount)

    injected = []

    if mode == "binned":
        # binned approach: apply multiplicative factors to provided density
        assert binned_kwargs is not None, "binned_kwargs required for mode='binned'."
        bins = np.asarray(binned_kwargs["bins"])
        density = np.asarray(binned_kwargs["density"]).astype(float)
        xcenters = 0.5 * (bins[:-1] + bins[1:])
        density_new = density.copy()
        for i in range(amount):
            c = centers[i]
            sigma = sigmas[i]
            if binned_kwargs.get("type", "gap") == "gap":
                depth = depths_arr[i]
                factor = 1.0 - depth * np.exp(-0.5 * ((xcenters - c)/sigma)**2)
            else:
                amp = amps_arr[i]
                factor = 1.0 + amp * np.exp(-0.5 * ((xcenters - c)/sigma)**2)
            density_new *= factor  # multiplicative modulation
            injected.append(dict(kind=binned_kwargs.get("type","gap"), center=float(c),
                                 sigma=float(sigma),
                                 depth=float(depths_arr[i]) if binned_kwargs.get("type","gap") else None,
                                 amp=float(amps_arr[i]) if not binned_kwargs.get("type","gap") else None))
        return density_new, injected

    # MODE == "point" (operate on catalogs)
    # We'll apply sequentially: for gaps we probabilistically remove points, for overdensities duplicate points with Poisson draws.
    phi1_work = phi1.copy()
    phi2_work = phi2.copy() if has_phi2 else None
    vlos_work = vlos.copy() if has_vlos else None

    for i in range(amount):
        c = centers[i]
        sigma = sigmas[i]
        depth = depths_arr[i]
        amp = amps_arr[i]

        # Gaussian profile (value from 0..1)
        # For a gap (underdensity): keep_prob = 1 - depth * exp(...)
        # For an overdensity: extra_expectation = amp * exp(...)
        g = np.exp(-0.5 * ((phi1_work - c)/sigma)**2)

        # --- gap (probabilistic removal) ---
        # compute keep probability per star
        keep_prob = 1.0 - depth * g
        # safety clamp to [0,1]
        keep_prob = np.clip(keep_prob, 0.0, 1.0)

        # sample uniform to decide which stars to keep
        u = rng.random(size=keep_prob.size)
        keep_mask = (u < keep_prob)

        # apply removal
        phi1_work = phi1_work[keep_mask]
        if has_phi2:
            phi2_work = phi2_work[keep_mask]
        if has_vlos:
            vlos_work = vlos_work[keep_mask]

        # --- overdensity (Poisson duplication) ---
        # For overdensity we add additional stars: for each remaining star, expected extra count = amp * g
        # Draw k ~ Poisson(amp * g) and duplicate point k times.
        # Note: we only apply overdensity in addition to gap if amp>0
        if amp > 1e-12:
            # recompute g on current catalog (after the gap)
            g2 = np.exp(-0.5 * ((phi1_work - c)/sigma)**2)
            lam = amp * g2  # expected extra count per star (can be <1)
            # draw Poisson for each star
            extra_counts = rng.poisson(lam)
            if extra_counts.sum() > 0:
                # indices with extra
                idx_extra = np.nonzero(extra_counts)[0]
                # build arrays to append
                phi1_extra = np.repeat(phi1_work[idx_extra], extra_counts[idx_extra])
                if has_phi2:
                    phi2_extra = np.repeat(phi2_work[idx_extra], extra_counts[idx_extra])
                if has_vlos:
                    vlos_extra = np.repeat(vlos_work[idx_extra], extra_counts[idx_extra])
                # append extras
                phi1_work = np.concatenate([phi1_work, phi1_extra])
                if has_phi2:
                    phi2_work = np.concatenate([phi2_work, phi2_extra])
                if has_vlos:
                    vlos_work = np.concatenate([vlos_work, vlos_extra])

        injected.append(dict(kind="gaussian_gap_plus_over", center=float(c),
                             sigma=float(sigma),
                             depth=float(depth),
                             amp=float(amp)))
    # final: optionally shuffle to avoid sorted blocks
    order = rng.permutation(len(phi1_work))
    phi1_new = phi1_work[order]
    phi2_new = phi2_work[order] if has_phi2 else None
    vlos_new = vlos_work[order] if has_vlos else None

    return phi1_new, phi2_new, vlos_new, injected


In [ ]:
size = 50000
mu, phi2_sigma = 0, 1 # mean and standard deviation
vlos_sigma = 5
rng = np.random.default_rng()
phi1 = rng.uniform(-300, 300, size)
phi2 = rng.normal(mu, phi2_sigma, size)
vlos = rng.normal(mu, vlos_sigma, size)

amount = 4
phi1_low, phi2_low, vlos_low, _ = inject_substructure(phi1, phi2=phi2, vlos=vlos, amount=100, scale=100)
phi1_mid, phi2_mid, vlos_mid, _ = inject_substructure(phi1, phi2=phi2, vlos=vlos, amount=100, scale=10)
phi1_high, phi2_high, vlos_high, _ = inject_substructure(phi1, phi2=phi2, vlos=vlos, amount=100, scale=1)


S_low = (StellarStream.from_catalog(phi1_low, phi2_low, vlos_low, 'low stream'))
     # .select("restrict_phi1", phi1_lim=(-100, 0))
     # .select("restrict_phi2", phi2_lim=6.0))


S_mid = (StellarStream.from_catalog(phi1_mid, phi2_mid, vlos_mid, 'mid stream'))
     # .select("restrict_phi1", phi1_lim=(-100, 0))
     # .select("restrict_phi2", phi2_lim=6.0))


S_high = (StellarStream.from_catalog(phi1_high, phi2_high, vlos_high, 'high stream'))
     # .select("restrict_phi1", phi1_lim=(-100, 0))
     # .select("restrict_phi2", phi2_lim=6.0))

S = (StellarStream.from_catalog(phi1, phi2, vlos, 'base stream'))
          # .select("restrict_phi1", phi1_lim=(-100, 0))
          # .select("restrict_phi2", phi2_lim=6.0))



S_low.plot_stream()
S_mid.plot_stream()
S_high.select("restrict_phi1", phi1_lim=(-100, 0)).plot_stream()
plt.show()

# bins_low, results_low = substructures(*S_low.density_phi1())
# bins_mid, results_mid = substructures(*S_mid.density_phi1())
# bins_high, results_high = substructures(*S_high.density_phi1())
# plt.plot(bins_low, results_low, label='low freq')
# plt.plot(bins_mid, results_mid, label='mid freq')
# plt.plot(bins_high, results_high, label='high freq')
# plt.legend()
# plt.show()


StellarStream.plot_power_spectrum_denoised(S_low, S_mid, S_high, S, precision=0.2)
StellarStream.plot_power_spectrum(S_low, S_mid, S_high, S, precision=0.2, window='hann')


In [ ]:
from scipy.integrate import simpson
from scipy.ndimage import gaussian_filter1d

def substructures_1(bins: npt.NDArray, dens: npt.NDArray) -> npt.NDArray:
    widths = np.diff(bins)
    bin_width = widths[0]  # uniform because linspace

    bins = []
    results = []
    for i in range(1, 300):
        sigma_bins = float(i/100) / bin_width
        bins.append(sigma_bins * bin_width)
        dens_new = gaussian_filter1d(dens, sigma=sigma_bins)
        results.append(simpson(np.abs(dens_new - dens)))
        dens = dens_new
        
    bins = np.array(bins)
    results = np.array(results)/simpson(results, x=bins)  # Normalize by the maximum frequency
    return bins, results

def substructures_2(bins: npt.NDArray, dens: npt.NDArray) -> npt.NDArray:
    widths = np.diff(bins)
    bin_width = widths[0]  # uniform because linspace

    bins = []
    results = []
    for i in range(1, 300):
        sigma_bins = float(i/100) / bin_width
        bins.append(sigma_bins * bin_width)
        dens_new = gaussian_filter1d(dens, sigma=sigma_bins)
        results.append(simpson(np.abs(dens_new - dens)))
        
    bins = np.array(bins)[:-1]
    results = np.diff(np.array(results))
    
    return bins, results/simpson(results, x=bins)  # Normalize

import numpy as np

def fourier_substructures(bins: np.ndarray, dens: np.ndarray):
    """
    Decompose a signal into substructure strength at each scale using Fourier analysis.

    Parameters
    ----------
    bins : np.ndarray
        The x-axis values (uniform spacing assumed).
    dens : np.ndarray
        The density values.

    Returns
    -------
    freqs : np.ndarray
        The Fourier frequencies (cycles per unit of bins).
    power_norm : np.ndarray
        The normalized power spectrum (substructure strength per scale).
    """
    # Uniform bin spacing
    dx = bins[1] - bins[0]
    
    # FFT of the signal
    fft_vals = np.fft.rfft(dens - np.mean(dens))  # remove mean to ignore DC component
    power = np.abs(fft_vals)**2  # power spectrum
    
    # Corresponding frequencies (cycles per unit length)
    freqs = np.fft.rfftfreq(len(dens), d=dx)
    
    # Normalize so the integral over frequency is 1
    norm = np.trapz(power, freqs)
    power_norm = power / norm if norm > 0 else power
    
    return freqs, power_norm



# bins_1, results_1 = substructures_1(*S_high.density_phi1())
# bins_2, results_2 = substructures_2(*S_high.density_phi1())

bins, results = substructures_2(*S.density_phi1())
bins_low, results_low = substructures_2(*S_low.density_phi1())
bins_mid, results_mid = substructures_2(*S_mid.density_phi1())
bins_high, results_high = substructures_2(*S_high.density_phi1())

# plt.plot(bins_1, results_1, label='1')
# plt.plot(bins_2, results_2, label='2')

plt.plot(bins_low, results_low, label='low freq')
plt.plot(bins_mid, results_mid, label='mid freq')
plt.plot(bins_high, results_high, label='high freq')

# plt.xscale('log')
# plt.yscale('log')
plt.legend()


print(f"low mean: {np.mean(results_low)}")
print(f"mid mean: {np.mean(results_mid)}")
print(f"high mean: {np.mean(results_high)}")

In [ ]:
_, _, freq_high, ps_high = compute_power_spectrum(phi1_high)
_, _, freq_mid, ps_mid = compute_power_spectrum(phi1_mid)
_, _, freq_low, ps_low = compute_power_spectrum(phi1_low)

plt.plot(freq_high[1:], ps_high[1:])
plt.plot(freq_mid[1:], ps_mid[1:])
plt.plot(freq_low[1:], ps_low[1:])
plt.xscale('log')
plt.yscale('log')

In [ ]:
size = 50000
mu, phi2_sigma = 0, 1 # mean and standard deviation
vlos_sigma = 5
rng = np.random.default_rng(seed=42)
phi1 = rng.uniform(-180, 180, size)
phi2 = rng.normal(mu, phi2_sigma, size)
vlos = rng.normal(mu, vlos_sigma, size)

amount = 4
phi1_low, phi2_low, vlos_low, _ = inject_substructure(phi1, phi2=phi2, vlos=vlos, amount=50, scale=1e-1)
phi1_mid, phi2_mid, vlos_mid, _ = inject_substructure(phi1, phi2=phi2, vlos=vlos, amount=50, scale=1e0)
phi1_high, phi2_high, vlos_high, _ = inject_substructure(phi1, phi2=phi2, vlos=vlos, amount=50, scale=1e1)

tile = 10
view = 360
add_low = np.concatenate([np.repeat(i * 360, len(phi1_low)) for i in range(tile)])
phi1_low, phi2_low, vlos_low = np.tile(phi1_low, (tile,)) + add_low, np.tile(phi2_low, (tile,)), np.tile(vlos_low, (tile,))

add_mid = np.concatenate([np.repeat(i * 360, len(phi1_mid)) for i in range(tile)])
phi1_mid, phi2_mid, vlos_mid = np.tile(phi1_mid, (tile,)) + add_mid, np.tile(phi2_mid, (tile,)), np.tile(vlos_mid, (tile,))

add_high = np.concatenate([np.repeat(i * 360, len(phi1_high)) for i in range(tile)])
phi1_high, phi2_high, vlos_high = np.tile(phi1_high, (tile,)) + add_high, np.tile(phi2_high, (tile,)), np.tile(vlos_high, (tile,))


S_low = (StellarStream.from_catalog(phi1_low, phi2_low, vlos_low, 'low stream'))
     # .select("restrict_phi1", phi1_lim=(-100, 0))
     # .select("restrict_phi2", phi2_lim=3.0))


S_mid = (StellarStream.from_catalog(phi1_mid, phi2_mid, vlos_mid, 'mid stream'))
     # .select("restrict_phi1", phi1_lim=(-100, 0))
     # .select("restrict_phi2", phi2_lim=3.0))


S_high = (StellarStream.from_catalog(phi1_high, phi2_high, vlos_high, 'high stream'))
     # .select("restrict_phi1", phi1_lim=(-100, 0))
     # .select("restrict_phi2", phi2_lim=3.0))

S = (StellarStream.from_catalog(phi1, phi2, vlos, 'base stream'))
          # .select("restrict_phi1", phi1_lim=(-100, 0))
          # .select("restrict_phi2", phi2_lim=3.0))



S_low.plot_stream()
S_mid.plot_stream()
S_high.plot_stream()

In [ ]:
bins, norm_dens = S.density_phi1(smooth=True, interpolate=True, precision=1)
_, high_dens = S_high.density_phi1(smooth=True, interpolate=True, precision=1)
_, mid_dens = S_mid.density_phi1(smooth=True, interpolate=True, precision=1)
_, low_dens = S_low.density_phi1(smooth=True, interpolate=True, precision=1)

diff_high = np.abs(norm_dens - high_dens)
diff_mid = np.abs(norm_dens - mid_dens)
diff_low = np.abs(norm_dens - low_dens)

plt.figure(figsize=(10, 6), dpi=300)
plt.scatter(bins, diff_high, label='High Density Difference')
plt.scatter(bins, diff_mid, label='Mid Density Difference')
plt.scatter(bins, diff_low, label='Low Density Difference')
plt.legend()
plt.grid()
plt.show()


In [ ]:
bins, freqs = S.power_spectrum(smooth=True, interpolate=True, precision=1)
_, low_freq = S_low.power_spectrum(smooth=True, interpolate=True, precision=1)
_, mid_freq = S_mid.power_spectrum(smooth=True, interpolate=True, precision=1)
_, high_freqs = S_high.power_spectrum(smooth=True, interpolate=True, precision=1)


# bins, freqs = S.power_spectrum()
# _, low_freq = S_low.power_spectrum()
# _, mid_freq = S_mid.power_spectrum()
# _, high_freqs = S_high.power_spectrum()

mask = (bins > 0)

diff_high = np.abs(freqs - high_freqs)
diff_high_normalized = diff_high / simpson(diff_high, x=bins)  # Normalize by the maximum frequency
diff_low = np.abs(freqs - low_freq)
diff_low_normalized = diff_low / simpson(diff_low, x=bins)  # Normalize by the maximum frequency
diff_mid = np.abs(freqs - mid_freq)
diff_mid_normalized = diff_mid / simpson(diff_mid, x=bins)  # Normalize by the maximum frequency

plt.figure(figsize=(10, 6), dpi=300)
plt.plot(bins[mask], diff_high_normalized[mask], label='High Frequency Difference')
plt.plot(bins[mask], diff_low_normalized[mask], label='Low Frequency Difference')
plt.plot(bins[mask], diff_mid_normalized[mask], label='Mid Frequency Difference')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.grid()
plt.show()

In [ ]:
np.exp(10**-1)

In [ ]:
import numpy as np
from numpy.fft import rfft, rfftfreq
from scipy.signal import get_window, welch
from numpy.typing import ArrayLike

def power_spectrum_denoised(
    phi1: ArrayLike,
    y: ArrayLike,
    *,
    grid_ddeg: float = 0.1,          # uniform bin width in degrees
    detrend: str = "poly3",          # "none"|"poly2"|"poly3"
    window: str = "tukey",           # "tukey"|"hann"
    tukey_alpha: float = 0.25,
    method: str = "welch",           # "welch"|"fft"|"lomb" (lomb not shown here)
    n_segments: int = 6,             # Welch: 4–8 is typical for ~100°
    overlap: float = 0.5,            # 50% overlap
    median_average: bool = True,     # median Welch
    subtract_highk_floor: bool = True,
    return_errors: bool = True,
):
    """
    Returns:
        f_cpd : frequencies [cycles/degree]
        ps    : one-sided PSD with energy-preserving normalization
        err   : jackknife error (if return_errors=True), else None
    """
    phi1 = np.asarray(phi1)
    y    = np.asarray(y)

    # 1) Uniform grid (inverse-variance weighting optional)
    phi_min, phi_max = np.min(phi1), np.max(phi1)
    edges = np.arange(phi_min, phi_max + grid_ddeg, grid_ddeg)
    centers = 0.5*(edges[1:] + edges[:-1])
    # simple binning:
    idx = np.digitize(phi1, edges) - 1
    mask = (idx >= 0) & (idx < len(centers))
    bsum = np.bincount(idx[mask], weights=y[mask], minlength=len(centers))
    bcnt = np.bincount(idx[mask], minlength=len(centers))
    dens = np.zeros_like(centers)
    nz = bcnt > 0
    dens[nz] = bsum[nz] / bcnt[nz]
    # fill small gaps by linear interp (optional):
    if np.any(~nz):
        dens = np.interp(centers, centers[nz], dens[nz])

    # normalize to mean 1 (optional but common for density)
    dens /= np.mean(dens)

    # 2) Detrend
    if detrend == "poly2":
        X = np.vstack([np.ones_like(centers), centers, centers**2]).T
        beta = np.linalg.lstsq(X, dens, rcond=None)[0]
        trend = X @ beta
        resid = dens - trend
    elif detrend == "poly3":
        X = np.vstack([np.ones_like(centers), centers, centers**2, centers**3]).T
        beta = np.linalg.lstsq(X, dens, rcond=None)[0]
        trend = X @ beta
        resid = dens - trend
    else:
        resid = dens - np.mean(dens)

    # 3) Window (apodize)
    if window == "tukey":
        w = get_window(("tukey", tukey_alpha), len(resid), fftbins=True)
    else:
        w = get_window("hann", len(resid), fftbins=True)
    x = resid * w

    # 4) PSD
    dphi = grid_ddeg  # sampling in degrees
    if method == "fft":
        # one-sided periodogram with energy-preserving normalization
        n = len(x)
        X = rfft(x)
        f = rfftfreq(n, d=dphi)  # cycles per degree
        ps = (2.0 * (np.abs(X)**2) * dphi / (n))  # one-sided; factor 2 except DC/Nyquist handled by rfft
        # fix DC/Nyquist normalization
        ps[0] /= 2.0
        if n % 2 == 0:
            ps[-1] /= 2.0
        spectra = None
    else:
        # Welch with robust (median) combine
        nperseg = int(np.floor(len(x) / (1 + (1-overlap) * (n_segments-1))))
        nperseg = max(nperseg, 32)
        noverlap = int(overlap * nperseg)
        f, ps_w = welch(
            x, fs=1.0/dphi, window=window if window!="tukey" else ("tukey", tukey_alpha),
            nperseg=nperseg, noverlap=noverlap, detrend=False, return_onesided=True, scaling="density"
        )
        # welch(scaling="density") already returns energy-preserving (power per cpd)
        ps = ps_w
        spectra = None  # for simple jackknife we’ll re-run internally on segments if needed

    # 5) Subtract white-noise floor (empirical)
    if subtract_highk_floor:
        hi = f > 0.7*np.max(f)  # top 30% of band
        if np.any(hi):
            floor = np.median(ps[hi])
            ps = np.clip(ps - floor, a_min=0.0, a_max=None)

    # 6) Jackknife errors over Welch segments (quick approximation)
    err = None
    if return_errors and method == "welch":
        # redo Welch but extract segment spectra and jackknife
        # (simple approximation: split into K blocks without overlap for DOF)
        K = max(4, n_segments//1)
        seg_len = len(x)//K
        seg_ps = []
        for k in range(K):
            seg = x[k*seg_len:(k+1)*seg_len]
            if len(seg) < 32: continue
            fk, Pk = welch(
                seg, fs=1.0/dphi, window=window if window!="tukey" else ("tukey", tukey_alpha),
                nperseg=min(len(seg), nperseg), noverlap=0, detrend=False, return_onesided=True, scaling="density"
            )
            seg_ps.append(Pk)
        if len(seg_ps) >= 2:
            seg_ps = np.array(seg_ps)
            # match frequency grid
            if fk.shape == f.shape:
                mu = np.mean(seg_ps, axis=0)
                jk = []
                for i in range(len(seg_ps)):
                    mu_i = np.mean(np.delete(seg_ps, i, axis=0), axis=0)
                    jk.append(mu_i)
                jk = np.array(jk)
                # jackknife variance
                err = np.sqrt((len(seg_ps)-1) * np.mean((jk - mu)**2, axis=0))
            else:
                err = None

    return f, ps, err


In [ ]:
freq_low, ps_low, err_low = power_spectrum_denoised(phi1_low, phi2_low)
freq_mid, ps_mid, err_mid = power_spectrum_denoised(phi1_mid, phi2_mid)
freq_high, ps_high, err_high = power_spectrum_denoised(phi1_high, phi2_high)

plt.plot(freq_low[1:], ps_low[1:], label='low freq', alpha=0.7)
plt.plot(freq_mid[1:], ps_mid[1:], label='mid freq', alpha=0.7)
plt.plot(freq_high[1:], ps_high[1:], label='high freq', alpha=0.7)   
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
on = True
freq_low, ps_low, err_low = power_spectrum_denoised(*S_low.density_phi1(smooth=on, interpolate=on, precision=1))
freq_mid, ps_mid, err_mid = power_spectrum_denoised(*S_mid.density_phi1(smooth=on, interpolate=on, precision=1))
freq_high, ps_high, err_high = power_spectrum_denoised(*S_high.density_phi1(smooth=on, interpolate=on, precision=1))

plt.plot(freq_low[1:], ps_low[1:], label='low freq', alpha=0.7)
plt.plot(freq_mid[1:], ps_mid[1:], label='mid freq', alpha=0.7)
plt.plot(freq_high[1:], ps_high[1:], label='high freq', alpha=0.7)   
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
print(ps_low.mean())
print(ps_mid.mean())
print(ps_high.mean())